In [5]:
import pandas as pd
import numpy as np

class preproccessing_func:
    
    ## functionto drop the columns that have a percentage of NA above the defined
    def limpiar_data(data,pct_filtrar=0.2):
        ## make a dataframe that resumes the column name with the pct of NaNs
        variables_temp=[[i,data[i].isna().sum()/len(data)] for i in data.columns]
        variables_temp=pd.DataFrame(variables_temp,columns=['variable','pct_filter'])

        # columns that have a nan pct greater than the parameter
        columns=variables_temp[variables_temp['pct_filter']<pct_filtrar]['variable'].tolist()

        #filter the columns that we want
        data=data[columns]

        return data
    
    ## function to get the variables with the highest correlations
    def correlations_treatment(data,pct_correlation=0.7):
        empty_df=pd.DataFrame()
        temp_corr=data.corr()
        for i in data.columns:
            col_temp=temp_corr[np.abs(temp_corr[i])>pct_correlation][i]
            if len(col_temp)==1:
                1+1
            else:
                empty_df=pd.concat([empty_df,col_temp],axis=1)
        return empty_df
    
    ## function to get the columns that cointains other columns information
    def corr_cought(bool_list):
        for i in range(len(bool_list)):
            counter=0
            if bool_list.iloc[0:(i+1)].sum()==1:
                break
        return i
    
    ################################################################
    ## define a function to convert numeric variables to categorical if they are concetrated
    def concentrated_transform(dataframe,treshold_pct=0.97,exclude_vars=[]):
        data=dataframe.copy(deep=True)

        ## estimate the Hirshman-Herdinfahl index for each variable.
        columns_list=[] ## columns to be categorized.
        for i in data.columns:
            temp_array=data[i].value_counts()/data[i].sum()
            if ((sum(temp_array>treshold_pct)==1) & (i not in exclude_vars)):
                columns_list.append([i,temp_array[temp_array>treshold_pct].index[0]])
        return columns_list
    
    ## function to treat outliers no matter the distribution of the data
    def outliers_treatment(dataset,variables='todas',## we can pass todas so it takes all the data frame or the list of variables we want to use
                           variables_exclude=[],## we can exclude some variables from the dataset
                           perc_step=2.5, ## how many percent will be reach bin when dividing the dataset. 
                           correction_type='replace',## it can be replace or exlude depending on the method you want to use
                           exclude_negatives=True, ## if you want to replace the negatives with missing values
                           methodology='percentile', ## methodology could be "percentile" or "std"
                           std_integer=2): ## if methodology == "std" decide how much std would you accept among the "normal" data
        data=dataset.copy(deep=True)
        if exclude_negatives:
            data[(data<0)]=None
        if variables=='todas':
            columnas_convertir=data.columns.tolist()
            [columnas_convertir.remove(i) for i in variables_exclude]
        else:
            columnas_convertir=variables
        if correction_type=='replace':
            for variable in columnas_convertir:
                if methodology=='percentile':
                    ## sacamos los percentiles de la distribución, asi como el minimo y el maximo de la serie
                    tabla_percentiles=np.percentile(data[variable].dropna(),np.arange(0,101,perc_step),interpolation='midpoint').tolist()
                    ## miramos en que percentiles se presenta el rango mas alto de la distribiución
                    tabla_diferencias=np.diff(tabla_percentiles)

                    ## Sacamos los dos percentiles que pertenecen a los rangos mas amplios
                    argsort_differences=tabla_diferencias.argsort()
                    list_int_index=argsort_differences[-2:].tolist()

                    ## miramos los rangos de percentiles que mas separados estan en la distribución

                    array_percentiles=[]
                    for index in list_int_index:
                        array_percentiles.append(tabla_percentiles[index:index+2])
                    if (abs(list_int_index[0]-list_int_index[1])==1):
                        ## primer caso en el que los dos estan pegados en la punta derecha
                        if bool(np.isin(np.max(argsort_differences),list_int_index)):
                            data.loc[(data[variable]>=np.min(array_percentiles)),variable]=np.min(array_percentiles)
                        ## Segundo caso en el que los dos estan pegados en la cola izquierda
                        elif bool(np.isin(np.min(argsort_differences),list_int_index)):
                            data.loc[(data[variable]>=np.max(array_percentiles)),variable]=np.max(array_percentiles)
                        ## tercer caso en el que estan pegados en el medio
                        else:
                            ## en este caso si importa el orden para saber si asignarle el maximo o el minimo del percentil
                            if list_int_index[0]>list_int_index[1]:
                                data.loc[(data[variable].between(array_percentiles[0][0],array_percentiles[0][1])),variable]=np.max(array_percentiles[0])
                                data.loc[(data[variable].between(array_percentiles[1][0],array_percentiles[1][1])),variable]=np.min(array_percentiles[1])
                            elif list_int_index[1]>list_int_index[0]:
                                data.loc[(data[variable].between(array_percentiles[0][0],array_percentiles[0][1])),variable]=np.min(array_percentiles[0])
                                data.loc[(data[variable].between(array_percentiles[1][0],array_percentiles[1][1])),variable]=np.max(array_percentiles[1])
                    else:
                        for array in array_percentiles:
                            if array[0]==np.min(tabla_percentiles):
                                data.loc[(data[variable].between(array[0],array[1])),variable]=array[1]
                            elif array[1]==np.max(tabla_percentiles):
                                data.loc[(data[variable].between(array[0],array[1])),variable]=array[0]
                            else:
                                medium=(array[1]+array[0])/2
                                data.loc[(data[variable].between(array[0],medium)),variable]=array[0]
                                data.loc[(data[variable].between(medium,array[1])),variable]=array[1]
                elif methodology=='std':
                    ## Media y desviacion standar de la serie
                    desviacion=data[variable].std()
                    media=data[variable].mean()

                    # Reemplazamos la cola izquierda por la media -2 vecesa la desviación
                    data.loc[(data[variable]<(media-std_integer*desviacion)),variable]=media-std_integer*desviacion

                    # Hacemos lo mismo para la cola derecha
                    data.loc[(data[variable]>(media+std_integer*desviacion)),variable]=media+std_integer*desviacion

        elif correction_type=='exclude':
            for variable in data.columns:
                if methodology=='percentile':
                    print(variable)
                    ## sacamos los percentiles de la distribución, asi como el minimo y el maximo de la serie
                    tabla_percentiles=np.percentile(data[variable].dropna(),np.arange(0,101,perc_step),interpolation='midpoint').tolist()
                    ## miramos en que percentiles se presenta el rango mas alto de la distribiución
                    tabla_diferencias=np.diff(tabla_percentiles)

                    ## Sacamos los dos percentiles que pertenecen a los rangos mas amplios
                    argsort_differences=tabla_diferencias.argsort()
                    list_int_index=argsort_differences[-2:].tolist()

                    ## miramos los rangos de percentiles que mas separados estan en la distribución

                    array_percentiles=[]
                    for index in list_int_index:
                        array_percentiles.append(tabla_percentiles[index:index+2])
                    data.drop(data[data[variable].between(array_percentiles[0][0],array_percentiles[0][1])].index)
                    data.drop(data[data[variable].between(array_percentiles[0][0],array_percentiles[0][1])].index)
                elif methodology=='std':
                    ## Media y desviacion standar de la serie
                    desviacion=data[variable].std()
                    media=data[variable].mean()

                    # Reemplazamos la cola izquierda por la media -2 vecesa la desviación
                    data.drop(data.loc[(data[variable]<(media-std_integer*desviacion)),variable].index)

                    # Hacemos lo mismo para la cola derecha
                    data.drop(data.loc[(data[variable]>(media+std_integer*desviacion)),variable].index)

        return data
    
    ## function to extract columns that have atypical values
    ## list variables to treat because they are too concentred or have too many categories
    def categorical_variable_treatment(dataframe,concen_tresh=0.9,num_categories=20):
        columns_cat_list=[]
        data=dataframe.copy()
        for i in cat_df.columns:
            value_c_var=cat_df[i].value_counts()
            if ((len(value_c_var)>num_categories) | (((value_c_var/value_c_var.sum())>concen_tresh).sum()==1)):
                columns_cat_list.append([i,len(value_c_var),(value_c_var/value_c_var.sum()).max()])
        return columns_cat_list